<a href="https://colab.research.google.com/github/krisdanner/Aquisi-o-e-Convers-o-de-Coordenadas/blob/main/Aquisi%C3%A7%C3%A3o_e_Convers%C3%A3o_de_Coordenadas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import requests
import pandas as pd
import numpy as np
import openpyxl
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

url = 'https://data-live.flightradar24.com/zones/fcgi/feed.js?faa=1&bounds=0.977%2C-5.651%2C-57.118%2C-27.104&satellite=1&mlat=1&flarm=1&adsb=1&gnd=1&air=1&vehicles=1&estimated=1&maxage=14400&gliders=1&stats=1'

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.128 Safari/537.36'}

r = requests.get(url, headers=headers)
data = json.loads(r.content)

codigo = []
aviao = []
dados = []

def geo2cart(lat,lon,h):
  a = 6378137; e = 0.0818191908426215
  N = a/math.sqrt(1-(e**2)*math.sin(lat)**2)
  X = (N+h)*math.cos(lat)*math.cos(lon)
  Y = (N+h)*math.cos(lat)*math.sin(lon)
  Z = (N*(1-e**2)+h)*math.sin(lat)
  return [X,Y,Z]


def cart2local(X,Y,Z,lat0,lon0,h0):
  [x0,y0,z0] = geo2cart(lat0,lon0,h0)
  A = [[-math.sin(lon0), math.cos(lon0), 0], [-math.sin(lat0)*math.cos(lon0), -math.sin(lat0)*math.sin(lon0), math.cos(lat0)], [math.cos(lat0)*math.cos(lon0), math.cos(lat0)*math.sin(lon0), math.sin(lat0)]]
  B = [[X-x0], [Y-y0], [Z-z0]]
  A = np.dot(A,B)
  x = A[0]
  y = A[1]
  z = A[2]
  return [x,y,z]

def local2aer(x,y,z):
  if y > 0:
    az = math.atan(x/y)
  if y < 0:
    az = math.atan(x/y)+180
  if y == 0 and x > 0:
    az = 90
  if y == 0 and x < 0:
    az = 270
  if az < 0:
    az = az+360
  el = math.atan((z*math.sin(az))/x)
  dist = z/math.sin(el)
  return [az,el,dist]

for x in data:
    codigo.append(x)

del codigo[0], codigo[0]
codigo.pop()

for x in codigo:
    dados.append(round((data[x][4])*0.3048, 2))
    dados.append(data[x][1])
    dados.append(data[x][2])
    aviao.append(data[x][16])

col = 'Altitude Latitude Longitude'.split()
lin = aviao
dados_1 = np.array(dados).reshape(len(aviao), 3)
dataf = pd.DataFrame(data=dados_1, index=lin, columns=col)
print(dataf)

pos = int(input('Digite a posição da aeronave: '))
qtd = int(input('Digite quantos dados de posição deseja obter: '))

t = 0

while t < qtd:
    r = requests.get(url, headers=headers)
    data = json.loads(r.content)
    dados2 = []

    dados2.append((data[codigo[pos]][4])*0.3048)
    dados2.append(data[codigo[pos]][1])
    dados2.append(data[codigo[pos]][2])

    cart = []
    cart = geo2cart(dados2[1],dados2[2],dados2[0])
    local = []
    local = cart2local(cart[0],cart[1],cart[2],-2.3310835100,-44.4208387800,58.826)
    aer = []
    aer = local2aer(local[0],local[1],local[2])
    print(aer)

    print(round(100 * t / qtd, 2), '%')
    t += 1

         Altitude  Latitude  Longitude
AZU4433  11887.20     -0.84     -52.12
AZU4463   7635.24     -2.74     -51.13
AD2004    9753.60     -2.48     -50.85
AZU4375   7010.40     -0.33     -49.85
AZU4241   9144.00     -3.34     -49.10
AZU4210   9578.34     -3.16     -48.50
TAM3003      0.00     -1.38     -48.47
AZU4231   8526.78     -2.92     -48.10
TAM3089   4876.80     -3.49     -44.61
AZU4563   3276.60     -5.19     -42.65
PTGAX      327.66     -4.52     -37.75
AZU4656   2446.02     -5.45     -37.19
CLX7601  10386.06     -0.97     -37.06
TAP079   12194.44      0.52     -36.12
AZU4091   7886.70     -4.76     -32.73
Digite a posição da aeronave: 1
Digite quantos dados de posição deseja obter: 20
[179.2143939579408, 0.048785352758315574, array([-14528788.15503154])]
0.0 %
[179.2143939579408, 0.048785352758315574, array([-14528788.15503154])]
5.0 %
[179.2143939579408, 0.048785352758315574, array([-14528788.15503154])]
10.0 %
[179.21947475591733, 0.049264225955622895, array([-13605922.248